In [10]:
import pandas as pd
import numpy as np
import pickle
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load dataset
git_link = 'https://github.com/geojoseph19/FakeFinder/raw/master/Datasets/Combined/insta_dataset.csv'
df = pd.read_csv(git_link)

# Preprocess data
numerical_cols = ['profile pic', 'nums/length username', 'fullname words', 'nums/length fullname', 'name==username', 'description length', 'external URL', 'private', '#posts', '#followers', '#follows']
X = df[numerical_cols]
y = df['fake']

# Normalize numerical variables
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# Save the StandardScaler to disk
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=40)

# Define LSTM model
model = Sequential()
model.add(LSTM(64, input_shape=(X_train.shape[1], 1)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train LSTM model
X_train_reshaped = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
model.fit(X_train_reshaped, y_train, epochs=10, batch_size=32)

# Save the trained LSTM model to disk
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

# Load the saved StandardScaler and LSTM model from disk
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

with open('model.pkl', 'rb') as f:
    model = pickle.load(f)

# Preprocess the input data using the saved StandardScaler
input_data = pd.DataFrame(np.array([1, 0.0, 2, 0.0, 0, 148, 0, 0, 6, 39, 12]).reshape(1,-1), columns=['profile pic', 'nums/length username', 'fullname words', 'nums/length fullname', 'name==username', 'description length', 'external URL', 'private', '#posts', '#followers', '#follows'])
input_data_normalized = scaler.transform(input_data)

# Make predictions using the saved LSTM model
input_data_reshaped = np.reshape(input_data_normalized, (input_data_normalized.shape[0], input_data_normalized.shape[1], 1))
y_pred = model.predict(input_data_reshaped)

# Evaluate LSTM model on test data
X_test_reshaped = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
loss, accuracy = model.evaluate(X_test_reshaped, y_test, batch_size=32)

print('Test loss:', loss)
print('Test accuracy:', accuracy)

print(y_pred)

Epoch 1/10
18/18 [==============================] - 2s 7ms/step - loss: 0.6736 - accuracy: 0.6241
Epoch 2/10
18/18 [==============================] - 0s 6ms/step - loss: 0.6221 - accuracy: 0.6547
Epoch 3/10
18/18 [==============================] - 0s 6ms/step - loss: 0.5837 - accuracy: 0.6637
Epoch 4/10
18/18 [==============================] - 0s 6ms/step - loss: 0.5501 - accuracy: 0.6978
Epoch 5/10
18/18 [==============================] - 0s 6ms/step - loss: 0.5106 - accuracy: 0.7446
Epoch 6/10
18/18 [==============================] - 0s 6ms/step - loss: 0.4484 - accuracy: 0.7806
Epoch 7/10
18/18 [==============================] - 0s 6ms/step - loss: 0.4118 - accuracy: 0.8129
Epoch 8/10
18/18 [==============================] - 0s 6ms/step - loss: 0.3916 - accuracy: 0.8435
Epoch 9/10
18/18 [==============================] - 0s 6ms/step - loss: 0.3766 - accuracy: 0.8327
Epoch 10/10
5/5 [==============================] - 1s 5ms/step - loss: 0.2515 - accuracy: 0.9286
Test loss: 0.25154349